# Provider Profile

In [1]:
# coding: utf-8
#
# This is the bare bones of the component 4.1.1 Model Manipulation
# It receives a JSON formatted instruction via the command line in
# the form:
#
# python model_manip.py
#
# Then send a string similar to the following in via std in:
# {"request_type": "provider_profile", "value": "Psych"}
#
# It responds via std out with a JSON string that includes the originating call
# and an additional response string (which in itself is a JSON string)

import pandas as pd
import json
import sys
import os

#directory = os.path.abspath(os.path.dirname(os.path.realpath(__file__)) + "/../test/data_input_component_CSV/")
directory = "../test/data_input_component_CSV/"

command="null"
provider_type="null"

In [2]:
def respond(response_msg,verb,object,error_msg=None):
    """Prints a JSON formatted service response to std out and exits the program

        Parameters
        ----------
        response_msg : str, mandatory
            The message included in the response attribute of the JSON out
        verb : str, mandatory
            The command that was received (if any)
        object: str, mandatory
            The target for the command that was received (if any)
        error_msg : str, optional
            An error message string
    """
    if (error_msg != None):
      response_msg = '{"error_msg":"' + error_msg + '"}'
    response = '{"request":{"request_type":"' + verb + '", "value": "' + object + '"},"response":' + response_msg + '}'
    print (response)
    sys.exit(0)

In [3]:
command_string = input("")

{"request_type": "provider_profile", "value": "Psych"}


In [4]:
# parse the command line argument into a JSON object
try:
    parsed_command = json.loads(command_string)
    provider_type = str(parsed_command["value"])
    command = str(parsed_command["request_type"])
    command
    provider_type
except Exception as e:
    respond(None,command,provider_type, "ERROR: Could not parse argument: "+str(e))

In [5]:
# check to see if command is understood
if command != "provider_profile":
    respond(None,command,provider_type, "ERROR: Unknown function call.")

In [6]:
# Next we import the relevant sheets from the Data Input Component (to begin with only three)
try:
    provider_list = pd.read_csv(directory + "/provider_list.csv")
    provider_supply = pd.read_csv(directory + "/provider_supply.csv")
    acute_service_prov_priority = pd.read_csv(directory + "/acute_service_prov_priority.csv")
except Exception as e:
    respond(None,command,provider_type, "ERROR: Could not read input files."+str(e))

In [7]:
provider_list.head(5)

,provider_name,provider_abbr,provider_mean_wage
0,Physician,Phys,"$177,958.00"
1,Physicial Assistant,PA,"$105,690.00"
2,Nurse Practitioner,NP,"$95,994.00"
3,Registered Nurse,RN,"$57,595.00"
4,Psychiatrist,Psych,"$204,194.00"


In [8]:
# First we create the header for the provider type with their full title and wage
provider_head_profile = provider_list.loc[provider_list['provider_abbr'] == provider_type]
# Then we check to see if the requested provider type is in the current provider list
if len(provider_head_profile.index) != 1:
    respond(None,command,provider_type, "ERROR: Provider type is not known.")
provider_head_profile

,provider_name,provider_abbr,provider_mean_wage
4,Psychiatrist,Psych,"$204,194.00"


In [9]:
provider_supply.head(5)

,provider_abbr,provider_county,provider_num,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Phys,Beaver County,4,NaN,NaN,NaN
1,Phys,Box Elder County,12,NaN,NaN,NaN
2,Phys,Cache County,14,NaN,NaN,NaN
3,Phys,Carbon County,3,NaN,NaN,NaN
4,Phys,Daggett County,0,NaN,NaN,NaN


In [10]:
# Second, we want to create the geographic profile of the provider type
# First we reduce the data frame to the three provider columns
provider_supply = provider_supply[['provider_abbr','provider_county','provider_num']]
# Then we reduce the data frame to just the rows relevant to the requested provider type
provider_supply1 = provider_supply.loc[provider_supply['provider_abbr'] == provider_type]
# Finally we strip out the provider abbrevialtion column as it is now redundant
provider_supply_profile = provider_supply1[['provider_county','provider_num']]

In [11]:
provider_supply_profile.head(5)

,provider_county,provider_num
148,Beaver County,0
149,Box Elder County,2
150,Cache County,22
151,Carbon County,1
152,Daggett County,0


In [12]:
acute_service_prov_priority.head(5)

,acute_encounter,acute_cateogry,acute_service,Phys,PA,NP,RN,Psych,LCSW,CMHC,MFT,PharmD,MA,Educ,Unnamed: 14
0,Abdominal pain,Measurements,BP,4,3,2,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,Abdominal pain,Measurements,Weight,4,3,2,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,Abdominal pain,Measurements,Temp,4,3,2,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,Abdominal pain,Patient History,Data Collection,4,3,2,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,Abdominal pain,Physical Exam,Problem Oriented Physical,4,3,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# For the final part of the profile, we change the provider type column in the service
# to provider mapping matrix to the priority column that we will output
acute_service_prov_priority.rename(columns={provider_type: 'priority'}, inplace=True)
# We then order the services from the most suitable for that provider down to the
# ones that they are not allowed to perform
acute_service_prov_priority=acute_service_prov_priority.sort_values(by=['priority','acute_encounter','acute_cateogry','acute_service'])
# We then reduce the output of the dataframe to the needed four columns
profile_services = acute_service_prov_priority[['priority','acute_encounter','acute_cateogry','acute_service']]
profile_services.head(10)

,priority,acute_encounter,acute_cateogry,acute_service
5,NaN,Abdominal pain,Lab,Orders
6,NaN,Abdominal pain,Lab,Results Consultation:
0,NaN,Abdominal pain,Measurements,BP
2,NaN,Abdominal pain,Measurements,Temp
1,NaN,Abdominal pain,Measurements,Weight
3,NaN,Abdominal pain,Patient History,Data Collection
7,NaN,Abdominal pain,Pharmacological Treatment/Mgmt,Orders/scripts
4,NaN,Abdominal pain,Physical Exam,Problem Oriented Physical
8,NaN,Abdominal pain,Visit wrap-up/care planning,Visit wrap-up/care planning
14,NaN,Chest pain,Lab,Orders


In [14]:
def strip_brackets(JSON_string):
    """Strips the brackets

        Parameters
        ----------
        JSON_string : str, mandatory
            The JSON string to strip the leading and trailing end brackets from
    """
    result = JSON_string.strip("[]")
    return result

In [15]:
# Finally we assemble the final JSON output from the three separate bits of the profile
out = '{"header":' + strip_brackets(provider_head_profile.to_json(orient='records')) + ','
out = out + '"supply":' + strip_brackets(provider_supply_profile.to_json(orient='records')) + ','
out = out + '"services":' + strip_brackets(profile_services.to_json(orient='records')) + '}'
# And issue them via std out and exit
#respond(out,command,provider_type)
respond(out,command,provider_type)

{"request":{"request_type":"provider_profile", "value": "Psych"},"response":{"header":{"provider_name":"Psychiatrist","provider_abbr":"Psych","provider_mean_wage":" $204,194.00 "},"supply":{"provider_county":"Beaver County","provider_num":0},{"provider_county":"Box Elder County","provider_num":2},{"provider_county":"Cache County","provider_num":22},{"provider_county":"Carbon County","provider_num":1},{"provider_county":"Daggett County","provider_num":0},{"provider_county":"Davis County","provider_num":16},{"provider_county":"Duchesne County","provider_num":1},{"provider_county":"Emery County","provider_num":0},{"provider_county":"Garfield County","provider_num":0},{"provider_county":"Grand County","provider_num":2},{"provider_county":"Iron County","provider_num":1},{"provider_county":"Juab County","provider_num":0},{"provider_county":"Kane County","provider_num":0},{"provider_county":"Millard County","provider_num":0},{"provider_county":"Morgan County","provider_num":0},{"provider_coun

SystemExit: 0

/Users/hopkira/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
